In [3]:
import json

import qa_extract
import pandas as pd
import plotly.io as pio

from text_conversion import clean_json, parse_data
from clusterization import llm_clusterization, plot_clusters

pd.set_option("display.max_colwidth", None)
pio.renderers.default = "notebook_connected"

SEED = 654321

In [4]:
res = qa_extract.main()

with open("data/qa.json", "r") as f:
    qa = json.load(f)

clean_qa = clean_json(qa)

In [5]:
clusters = llm_clusterization(clean_qa, "gpt-4-1106-preview", SEED)

with open('data/clusters.json', 'w',  encoding='utf-8') as f:
    json.dump(clusters, f, ensure_ascii=False, indent=4)
with open("data/clusters.json", "r") as f:
    clusters = json.load(f)

clusters

{'Проблемы с загрузкой и отображением фото': ['3',
  '30',
  '34',
  '38',
  '41',
  '48',
  '53',
  '79',
  '82',
  '84',
  '98',
  '113',
  '142',
  '158'],
 'Сбой в приложении и UI-оптимизация': ['7', '9', '68', '148'],
 'Потребность в обновлении приложения': ['13', '129', '153'],
 'Некорректная работа комментариев к заявкам': ['17', '65', '89', '93'],
 'Технические вопросы про использование приложения': ['25'],
 'Чрезмерный расход заряда батареи': ['27', '36'],
 'Вопросы по переносу заявок и навигации в приложении': ['75'],
 'Проблема с уведомлениями': ['59', '62', '133'],
 'Сложности со связью и звонками': ['72', '116', '125']}

In [6]:
cluster_map = {id_: name for name, ids in clusters.items() for id_ in ids}

rows = []
for item in qa:
    for key, value in item.items():
        if key.startswith('Answer'):
            question_id = key.split(" ")[1]
            affilate, division = parse_data(value, 'affilate', 'division')
            cluster_name = cluster_map.get(question_id, "Нет ответа")
            rows.append(
                {
                    "id": question_id,
                    "affilate": affilate,
                    "division": division,
                    "cluster": cluster_name,
                }
            )

df = pd.DataFrame(rows)
df

,id,affilate,division,cluster
0,3,None,None,Проблемы с загрузкой и отображением фото
1,7,МСК,ХД,Сбой в приложении и UI-оптимизация
2,9,None,None,Сбой в приложении и UI-оптимизация
3,13,None,None,Потребность в обновлении приложения
4,17,СПБ,СМ,Некорректная работа комментариев к заявкам
5,25,None,None,Технические вопросы про использование приложения
6,27,МСК,ХД,Чрезмерный расход заряда батареи
7,30,СПБ,СМ,Проблемы с загрузкой и отображением фото
8,34,None,None,Проблемы с загрузкой и отображением фото
9,36,None,None,Чрезмерный расход заряда батареи


In [8]:
df.to_csv("mrm_clusters.csv", index=False)
df = pd.read_csv("mrm_clusters.csv")

In [7]:
df.cluster.value_counts().to_frame().reset_index()

,cluster,count
0,Проблемы с загрузкой и отображением фото,14
1,Сбой в приложении и UI-оптимизация,4
2,Некорректная работа комментариев к заявкам,4
3,Потребность в обновлении приложения,3
4,Проблема с уведомлениями,3
5,Сложности со связью и звонками,3
6,Чрезмерный расход заряда батареи,2
7,Технические вопросы про использование приложения,1
8,Нет ответа,1
9,Вопросы по переносу заявок и навигации в приложении,1


In [8]:
cluster_plot = plot_clusters("Типы обращений", df, "cluster", 36, 30)
cluster_plot.show()

In [9]:
df.affilate.value_counts(dropna=False).to_frame().reset_index()

,affilate,count
0,МСК,14
1,None,13
2,СПБ,5
3,ЕКБ,3
4,НСК,1


In [10]:
affilate_plot = plot_clusters(
    "Филиалы обращений",
    df,
    "affilate",
    36,
    40,
    {None: "Не выгружен"}
)
affilate_plot.show()

In [11]:
df.division.value_counts(dropna=False).to_frame().reset_index()

,division,count
0,None,13
1,СМ,8
2,ХД,7
3,ТВ,3
4,МБС,2
5,УСТ,2
6,ПЛ,1


In [12]:
division_plot = plot_clusters(
    "Направления обращений", df, "division", 36, 40, {None: "Не выгружен"}
)
division_plot.show()